<a href="https://colab.research.google.com/github/ss8319/Colab-Notebooks/blob/main/Shamus_impersonator_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# セットアップ

In [ ]:
# ffmpegインストール
!apt-get install ffmpeg

# CUDAセッティング
import os
#os.environ["CUDA_HOME"] = "/usr/local/cuda-10.1"
os.environ["CUDA_HOME"] = "/usr/local/cuda-11"
!echo $CUDA_HOME

# githubからコードをコピー
!git clone https://github.com/cedro3/iPERCore.git
%cd /content/iPERCore/

# セットアッププログラムの実行
#!python setup.py develop
! pip install torch==1.7.0+cu110 torchvision==0.8.1+cu110 torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html
! pip install mmcv-full==1.2.2 -f https://download.openmmlab.com/mmcv/dist/cu110/torch1.7.0/index.html
! pip install git+https://github.com/open-mmlab/mmdetection.git@8179440ec5f75fe95484854af61ce6f6279f3bbc
! pip install git+https://github.com/open-mmlab/mmediting@d4086aaf8a36ae830f1714aad585900d24ad1156
! pip install git+https://github.com/iPERDance/neural_renderer.git@e5f54f71a8941acf372514eb92e289872f272653
! pip install tensorboardX>=2.1

# 重みのダウンロード
#! pip install --upgrade gdown
import gdown
gdown.download('https://drive.google.com/uc?id=1jpp_KytMplNNFA_IJSzzSWhvyqIdmnJv', 'assets/checkpoints.zip', quiet=False)
!unzip -o assets/checkpoints.zip -d assets/
!rm assets/checkpoints.zip

# サンプルのダウンロード
!wget -O assets/samples.zip  "https://download.impersonator.org/iper_plus_plus_latest_samples.zip"
!unzip -o assets/samples.zip -d  assets
!rm assets/samples.zip

# ディレクトリ移動
%cd /content/iPERCore/

# ライブラリーのインポート
import os
import os.path as osp
import platform
import argparse
import time
import sys
import subprocess
from IPython.display import HTML
from base64 import b64encode


Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.11-0ubuntu0.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.
/usr/local/cuda-11
Cloning into 'iPERCore'...
remote: Enumerating objects: 311, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 311 (delta 0), reused 0 (delta 0), pack-reused 307
Receiving objects: 100% (311/311), 18.90 MiB | 14.78 MiB/s, done.
Resolving deltas: 100% (41/41), done.
/content/iPERCore
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |███████████████████████▌        | 834.1 MB 1.2 MB/s eta 0:04:07tcmalloc: large alloc 1147494400 bytes == 0x3

Downloading...
From: https://drive.google.com/uc?id=1jpp_KytMplNNFA_IJSzzSWhvyqIdmnJv
To: /content/iPERCore/assets/checkpoints.zip
100%|██████████| 1.25G/1.25G [00:08<00:00, 149MB/s]


Archive:  assets/checkpoints.zip
   creating: assets/checkpoints/
  inflating: assets/__MACOSX/._checkpoints  
   creating: assets/checkpoints/neural_renders/
  inflating: assets/__MACOSX/checkpoints/._neural_renders  
   creating: assets/checkpoints/losses/
  inflating: assets/__MACOSX/checkpoints/._losses  
   creating: assets/checkpoints/restorers/
  inflating: assets/__MACOSX/checkpoints/._restorers  
   creating: assets/checkpoints/pose3d/
  inflating: assets/__MACOSX/checkpoints/._pose3d  
   creating: assets/checkpoints/pose2d/
  inflating: assets/__MACOSX/checkpoints/._pose2d  
   creating: assets/checkpoints/inpainting/
  inflating: assets/__MACOSX/checkpoints/._inpainting  
   creating: assets/checkpoints/detection/
  inflating: assets/__MACOSX/checkpoints/._detection  
   creating: assets/checkpoints/mattors/
  inflating: assets/__MACOSX/checkpoints/._mattors  
  inflating: assets/checkpoints/neural_renders/AttLWB-SPADE_id_G_2020-05-18.pth  
  inflating: assets/__MACOSX/chec

## 初期設定

In [ ]:
# the gpu ids
gpu_ids = "0"

# the image size
image_size = 512

# the default number of source images, it will be updated if the actual number of sources <= num_source
num_source = 2

# the assets directory. This is very important, please download it from `one_drive_url` firstly.
assets_dir = "/content/iPERCore/assets"

# the output directory.
output_dir = "./results"

# the model id of this case. This is a random model name.
# model_id = "model_" + str(time.time())

# # This is a specific model name, and it will be used if you do not change it.
# model_id = "axing_1"

# symlink from the actual assets directory to this current directory
work_asserts_dir = os.path.join("./assets")
if not os.path.exists(work_asserts_dir):
    os.symlink(osp.abspath(assets_dir), osp.abspath(work_asserts_dir),
               target_is_directory=(platform.system() == "Windows"))

cfg_path = osp.join(work_asserts_dir, "configs", "deploy.toml")


## Connect to Google Drive

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


# 1.ソースが正面1枚の場合の変換


In [ ]:
# This is a specific model name, and it will be used if you do not change it. This is the case of `trump`
model_id = "donald_trump_2"

# the source input information, here \" is escape character of double duote "
# src_path = "\"path?=/content/iPERCore/assets/samples/sources/donald_trump_2/00000.PNG,name?=donald_trump_2\""

# ref_path = "\"path?=/content/iPERCore/assets/samples/references/akun_1.mp4,"  \
#              "name?=akun_1," \
#              "pose_fc?=300\""
src_path = "\"path?=/content/iPERCore/assets/samples/sources/donald_trump_2/00000.PNG,name?=donald_trump_2\""


ref_path = "\"path?=/content/gdrive/MyDrive/Signapse and SIL/Salvadorian_1.mp4,"  \
             "name?=Salvadorian_1," \
             "pose_fc?=300\""

print(ref_path)

!python -m iPERCore.services.run_imitator  \
  --gpu_ids     $gpu_ids       \
  --num_source  $num_source    \
  --image_size  $image_size    \
  --output_dir  $output_dir    \
  --model_id    $model_id      \
  --cfg_path    $cfg_path      \
  --src_path    $src_path      \
  --ref_path    $ref_path

"path?=/content/gdrive/MyDrive/Signapse and SIL/Salvadorian_1.mp4,name?=Salvadorian_1,pose_fc?=300"
ffmpeg -y -i /content/gdrive/MyDrive/Signapse and SIL/Salvadorian_1.mp4 -ab 160k -ac 2 -ar 44100 -vn ./results/primitives/Salvadorian_1/processed/audio.mp3 -loglevel quiet
ffprobe -v error -select_streams v -of default=noprint_wrappers=1:nokey=1 -show_entries stream=r_frame_rate /content/gdrive/MyDrive/Signapse and SIL/Salvadorian_1.mp4
	Pre-processing: start...
----------------------MetaProcess----------------------
meta_input:
	path: /content/iPERCore/assets/samples/sources/donald_trump_2/00000.PNG
	bg_path: 
	name: donald_trump_2
primitives_dir: ./results/primitives/donald_trump_2
processed_dir: ./results/primitives/donald_trump_2/processed
vid_info_path: ./results/primitives/donald_trump_2/processed/vid_info.pkl
-------------------------------------------------------
	has_run_detector: True
	has_run_cropper: True
	has_run_3dpose: True
	has_find_front: True
	has_run_parser: True
	has_

In [ ]:
mp4 = open("./results/primitives/donald_trump_2/synthesis/imitations/donald_trump_2-Salvadorian_1.mp4", "rb").read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width="100%" height="100%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

# 2.ソースが正面＋背面の場合の変換
基本的に1.ソースが正面のみの場合と同じです。ソースのフォルダー内に、正面＋背面の画像を2つ入れて置くだけです。ソートした時に、正面の画像名が背面の画像名より先に来るようにして下さい。

In [ ]:
# This is a specific model name, and it will be used if you do not change it. This is the case of `trump`
model_id = "axing_1"

# the source input information, here \" is escape character of double duote "
src_path = "\"path?=/content/iPERCore/assets/samples/sources/axing_1,name?=axing_1\""


## the reference input information. There are three reference videos in this case.
# here \" is escape character of double duote "
ref_path = "\"path?=/content/iPERCore/assets/samples/references/dance01.mp4," \
             "name?=dance01," \
             "pose_fc?=300\""

print(ref_path)

!python -m iPERCore.services.run_imitator  \
  --gpu_ids     $gpu_ids       \
  --num_source  $num_source    \
  --image_size  $image_size    \
  --output_dir  $output_dir    \
  --model_id    $model_id      \
  --cfg_path    $cfg_path      \
  --src_path    $src_path      \
  --ref_path    $ref_path

In [ ]:
mp4 = open("./results/primitives/axing_1/synthesis/imitations/axing_1-dance01.mp4", "rb").read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width="100%" height="100%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

# ソースが正面＋背面＋背景の場合の変換
"bg_path?="で背景の画像を指定します。

In [ ]:
# This is a specific model name, and it will be used if you do not change it. This is the case of `trump`
model_id = "afan_6=ns=2"

# the source input information, here \" is escape character of double duote "
src_path = "\"path?=/content/iPERCore/assets/samples/sources/afan_6/afan_6=ns=2," \
             "name?=afan_6=ns=2," \
             "bg_path?=/content/iPERCore/assets/samples/sources/afan_6/IMG_7217.JPG\""

## the reference input information. There are three reference videos in this case.
# here \" is escape character of double duote "
ref_path = "\"path?=/content/iPERCore/assets/samples/references/dance02.mp4," \
             "name?=dance02," \
             "pose_fc?=300\""

print(ref_path)

!python -m iPERCore.services.run_imitator  \
  --gpu_ids     $gpu_ids       \
  --num_source  $num_source    \
  --image_size  $image_size    \
  --output_dir  $output_dir    \
  --model_id    $model_id      \
  --cfg_path    $cfg_path      \
  --src_path    $src_path      \
  --ref_path    $ref_path

"path?=/content/iPERCore/assets/samples/references/dance02.mp4,name?=dance02,pose_fc?=300"
ffmpeg -y -i /content/iPERCore/assets/samples/references/dance02.mp4 -ab 160k -ac 2 -ar 44100 -vn ./results/primitives/dance02/processed/audio.mp3 -loglevel quiet
ffprobe -v error -select_streams v -of default=noprint_wrappers=1:nokey=1 -show_entries stream=r_frame_rate /content/iPERCore/assets/samples/references/dance02.mp4
	Pre-processing: start...
----------------------MetaProcess----------------------
meta_input:
	path: /content/iPERCore/assets/samples/sources/afan_6/afan_6=ns=2
	bg_path: /content/iPERCore/assets/samples/sources/afan_6/IMG_7217.JPG
	name: afan_6=ns=2
primitives_dir: ./results/primitives/afan_6=ns=2
processed_dir: ./results/primitives/afan_6=ns=2/processed
vid_info_path: ./results/primitives/afan_6=ns=2/processed/vid_info.pkl
-------------------------------------------------------
----------------------MetaProcess----------------------
meta_input:
	path: /content/iPERCore/asse

In [ ]:
mp4 = open("./results/primitives/afan_6=ns=2/synthesis/imitations/afan_6=ns=2-dance02.mp4", "rb").read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width="100%" height="100%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

FileNotFoundError: ignored